In [ ]:
module Class
##################################################################
module MLP

const Lmax = 5 # maximum number of hidden layers
const Nmax = 300 # maximum number of neurons
const Nmin = 50  # minimum number of neurons
const epochs = 10
const maxevals = 10
const lr_min = 0.01# minimum learning rate
const lr_max = 0.5

using Knet, Hyperopt


function splitdata(x, y)
    n1, n2 = size(x)
    xtrn = x[:,1:4*n2÷5]
    xtst = x[:,4*n2÷5+1:n2]
    ytrn = y[:,1:4*n2÷5]
    ytst = y[:,4*n2÷5+1:n2]
    return  xtrn, ytrn, xtst, ytst
end 

function label2vec(l)
    global lu
    lu = unique(l) # unique label
    eltype(lu)<:Number && (lu = sort(lu))
    v = zeros(length(lu),length(l))
    for j = 1:size(v,2)
        i = find(lu .== l[j])[1]
        v[i,j] = 1
    end
    v
end

function vec2label(v)
    global lu
    l = Array{eltype(lu)}(1,size(v,2))
    for j = 1:size(v,2)
        i = findmax(v[:,j])[2]
        l[1,j] = lu[i]
    end
    l
end

function preprocess(x,y)
    global M, m
    M = maximum(x,2)
    m = minimum(x,2)
    x = (x.- m)./(M .- m .+ 1e-20)
    y = label2vec(y)
    return x,y
end

function predict(w,x)
    for i=1:2:length(w)
        x = w[i]*x .+ w[i+1]
        if i<length(w)-1
            x = relu(x) # max(0,x)
        end
    end
    return x
end

function loss(w,x,ygold)
    ypred = predict(w,x)
    ynorm = logp(ypred,1) # ypred .- log(sum(exp(ypred),1))
    -sum(ygold .* ynorm) / size(ygold,2)
end

lossgradient = grad(loss)

function train(w, dtrn; lr=.5, epochs=10)
    for epoch=1:epochs
        for (x,y) in dtrn
            g = lossgradient(w, x, y)
            for i in 1:length(w)
                w[i] -= lr * g[i]
            end
        end
    end
    return w
end

function err(w,dtrn)
    cost = 0.0
    for (x, ygold) in dtrn
        cost += loss(w,x,ygold)
    end
    cost
end

function accuracy(w, dtst)
    ncorrect = ninstance = 0
    for (x, ygold) in dtst
        ypred = predict(w, x)
        ncorrect += sum(ygold .* (ypred .== maximum(ypred,1)))
        ninstance += size(ygold,2)
    end
    return ncorrect/ninstance
end

function weights(data,h...; winit=0.1)
    x0,y0 = data[1]
    atype = typeof(MLP.dtrn[1][1])
    w = Any[]
    x = size(x0,1)
    for y in [h..., size(y0,1)]
        push!(w, convert(atype, winit*randn(y,x)))
        push!(w, convert(atype, zeros(y, 1)))
        x = y
    end
    return w  
end

function minibatch(x, y, batchsize; atype=Array{Float32})
    x = atype(x); y = atype(y)
    data = Any[]
    for i=1:batchsize:size(x,2)
        j=min(i+batchsize-1,size(x,2))
        push!(data, (x[:,i:j], y[:,i:j]))
    end
    return data
end

function objective(args)
    h, lr = args
    h = Int.(collect(h))
    
    global dtrn
    global dtst
    w = weights(dtrn,h...)

    loss = 1000
    for i=1:epochs
        train(w, dtrn; lr=lr, epochs=1)
        err = 1 - accuracy(w, dtst)
        if err < loss
            loss = err
        end
    end
    @printf("\nnlayer=%d,layers=%s,lr=%6.4f,loss=%6.4f\n",
    length(h),h,lr,loss)

    return Dict("loss" => loss, "status" => STATUS_OK, "model" => w)
end

export main
global dtrn, dtst, M, m, lu, atype
function main(x, y; batchsize=50, gpu=false)
    global dtrn, dtst, atype
    x, y = preprocess(x, y)
    xtrn, ytrn, xtst, ytst = splitdata(x, y)
    gpu == false ? (atype = Array{Float32}) : (atype = KnetArray{Float32})
    dtrn = minibatch(xtrn, ytrn, batchsize; atype=atype)
    dtst = minibatch(xtst, ytst, batchsize; atype=atype)

    trials = Trials()
    hps = [] # hyper-parameters
    opt = [] # option  
    
    for l = 1:Lmax
        push!(hps,quniform("h$l",Nmin,Nmax,50))
        push!(opt,(hps...))
    end
    
    best_args = fmin(objective,
    space=[choice("hidden",opt),uniform("lr", lr_min, lr_max)],
        algo=TPESUGGEST,
        maxevals=maxevals,
        trials = trials)
    
    best_loss, best_ind = findmin(losses(trials))    
    best_model = trials["results"][best_ind]["model"]
    
    best_model, best_args, best_loss
end

end # end of module MLP

##################################################################
module Boost

const max_depth_max = 100
const num_round_max = 10
const η_min = 0.1
const η_max = 1.0
const maxevals = 50

using XGBoost, Hyperopt

function splitdata(x, y)
    n1, n2 = size(x)
    xtrn = x[1:4*n1÷5, :]
    xtst = x[4*n1÷5+1:n1, :]
    ytrn = y[1:4*n1÷5]
    ytst = y[4*n1÷5+1:n1]
    return  xtrn, ytrn, xtst, ytst
end 

function label2int(l)
    global lu
    lu = unique(l) # unique label
    eltype(lu)<:Number && (lu = sort(lu))
    I = zeros(Int,l)
    for i in eachindex(I)
        I[i] = find(lu .== l[i])[1] - 1
    end
    I
end

function int2label(I)
    global lu
    I = Int.(I)
    l = zeros(I)
    for i in eachindex(I)
        l[i] = lu[I[i]+1]
    end
    l
end
    
function preprocess(x,y)  
    y = vec(y)
    y = label2int(y)
    return x,y
end


function objective(args)
    global xtrn, ytrn, xtst, ytst, nclass
    num_round,max_depth,η = args
    num_round = Int(num_round)
    max_depth = Int(max_depth)
    
    param = Dict("max_depth"=>max_depth,"eta"=>η, "num_class"=>nclass,
    "silent"=>2, "objective"=>"multi:softmax")
    bst = xgboost(xtrn, num_round, label=ytrn,param=param)
    preds = predict(bst, xtst)
    loss = mean(preds .!= ytst)
   
    @printf("\nnum_round=%d,max_depth=%d,η=%6.4f,loss=%6.4f\n",
    num_round,max_depth,η,loss)
    return Dict("loss" => loss, "status" => STATUS_OK,"model" => bst)
end

global xtrn, ytrn, xtst, ytst, nclass

export main
function main(x, y; batchsize=50, gpu=false)
    global xtrn, ytrn, xtst, ytst, nclass
    x, y = preprocess(x, y)
    xtrn, ytrn, xtst, ytst = splitdata(x, y)
    nclass = length(unique(y))
    
    trials = Trials()   
    best_args = fmin(objective,
    space=[quniform("num_round",1, num_round_max,1),
        quniform("max_depth",2, max_depth_max,1),
        uniform("eta", η_min, η_max)],
        algo=TPESUGGEST,
        maxevals=maxevals,
        trials = trials)
   
    best_loss, best_ind = findmin(losses(trials))    
    best_model = trials["results"][best_ind]["model"]
    
    best_model, best_args, best_loss
end

end # end of module Boost

##################################################################
export main
function main(x, y; o...)
    net, net_args, net_loss = MLP.main(x, y; o...)
    bst, bst_args, bst_loss = Boost.main(x, y; o...)
    net, net_args, net_loss, bst, bst_args, bst_loss
end

export predict
function predict(model::Boost.XGBoost.Booster, x)
    x = x'
    I = Boost.predict(model, x) # integer
    l = Boost.int2label(I) # label
end 
function predict(model::Array{Any}, x)
    x = (x.- MLP.m)./(MLP.M .- MLP.m .+ 1e-20)
    y = Array(MLP.predict(model, MLP.atype(x)))
    prob = exp(y)./sum(exp(y),1)
    preds = MLP.vec2label(y)
    return preds,prob
end 


end# end of module Class


In [ ]:
# import Class,MNIST
# xtrn, ytrn = MNIST.traindata()
# xtst, ytst = MNIST.testdata()
# x = [xtrn xtst]
# y = [ytrn;ytst] + 10
# # x = x[1:10,1:20]
# # y = y[1:20]
# net, net_args, net_loss, bst, bst_args, bst_loss = Class.main(x,y; gpu=true)

# net_preds, net_prob = Class.predict(net, x)
# bst_preds = Class.predict(bst, x);